# Modelo de clasificación para detección de sentimiento en tweets de migración

### Librerias útiles

##### Hugging Face
Se trata de una comunidad y plataforma de ciencia de datos que ofrece una gran cantidad de herramientas para la creación y evaluación de modelos de aprendizaje profundo. Entre ellas destacan [[Omer Mahmood @ Towards Data Science](https://towardsdatascience.com/whats-hugging-face-122f4e7eb11a#:~:text=Hugging%20Face%20is%20a%20community,(OS)%20code%20and%20technologies.)]:
* Herramientas que permiten a los usuarios construir, entrenar y desplegar modelos basados en código abierto
* Un lugar donde una amplia comunidad de científicos de datos, ingenieros de aprendizaje profundo e investigadores pueden reunirse para compartir ideas, obtener apoyo, contribuir a los proyectos e incluso compartir sus modelos entrenados o puros.

<figure>
    <img src="./assets/images/hug.png"
         alt="Hugging Face logo">
    <figcaption>Hugging Face logo</figcaption>
</figure>

Se usó un modelo entrenado para la clasificación de sentimiento en tweets en español, el cual se encuentra disponible en la plataforma de Hugging Face. El modelo usado fue [RoBERTuito-sentiment-analysis](https://arxiv.org/abs/2111.09453)


## Inferencia
Este notebook explica el proceso de inferencia de los tweets de migración utilizando el modelo de clasificación de sentimiento RoBERTuito de Hugging Face.

In [1]:
#Imports

#HuggingFace library
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TextClassificationPipeline

#data reading
import pandas as pd

### Cargando el modelo preentrenado

Dado que el modelo ya ha sido entrenado para clasificación de sentimiento en español, se puede cargar directamente desde la librería de Hugging Face. Para ello, se utiliza la función `AutoModelForSequenceClassification` de la librería `transformers` de Hugging Face.
No será necesario aplicar configuración alguna al modelo, al tokenizador tampoco debe añadirse ninguna configuración adicional.


In [2]:
#set model name
model_name = "pysentimiento/robertuito-sentiment-analysis"
#download model from huggingface
model = AutoModelForSequenceClassification.from_pretrained(
        model_name, num_labels=3)

#Load tokenizer
#Un tokenizer es un objeto que convierte una secuencia de caracteres en una secuencia de números.
#es una especie de filtro que prepara el texto para que el modelo lo pueda entender.
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Lectura de datos procesados

Para realizar una correcta inferencia de los datos deseados, éstos deberán ser preprocesados a través de las funciones descritas en el notebook `ETL.ipynb`. En este caso, los datos ya fueron procesados.

In [3]:
data = pd.read_csv('./assets/data/tweets_sentiment_infer.csv')
data.sample(5)

,text
1855,ENTONCES HAGA UN GRAN FAVOR A LA NAUGHTY NATIO...
1461,"@usuario @usuario Pinochet, Chile logró salir ..."
246,"@usuario en mi opinión, el manejo del tema mig..."
1693,Adivinen como entraban las armas a Chile : pue...
1621,Letanías Lauretanas A las ya conocidas el Papa...


### Predicción
Para la predicción de los datos, se utiliza la clase `TextClassificationPipeline` de la librería `transformers` de Hugging Face. Esta clase permite realizar la inferencia de los datos de manera sencilla, sin necesidad de realizar ninguna configuración adicional como en el caso del entrenamiento del modelo de xenofobia.

La clase `TextClassificationPipeline` recibe como parámetros el modelo preentrenado y el tokenizador. Además, se debe especificar el texto o textos que se deasean clasificar. En el caso de un único texto, el parametro será una cadena de caracteres. En el caso de múltiples textos, el parámetro será una lista de cadenas de caracteres.

Antes de proceder a la inferencia, se debe indicar al modelo que se desea realizar la inferencia en modo de evaluación. Para ello, se utiliza el método `eval()`.

In [4]:
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30002, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [5]:
#single text prediction
single_text_pred = TextClassificationPipeline(model=model, tokenizer=tokenizer, batch_size=16)
text = "Que opinas de la reformas Migratorias? emoji ojos emoji emoji ojos emoji mi buen amigo @usuario?\nYo pienso que DEbieron facilitar la entrada de los inmigrantes emoji persona de mostrador de información emoji emoji bandera brasil emoji emoji bandera costa rica emoji emoji bandera nicaragua emoji emoji bandera méxico emoji emoji bíceps flexionado emoji hashtag yo te apoyo eexx jaja. Página oficial del BID:url"
single_text_pred(text)

[{'label': 'NEU', 'score': 0.9595656394958496}]

In [6]:
data['sentiment'] = single_text_pred(data.text.tolist())

Se muestran cinco ejemplos de los datos clasificados por el modelo. Se muestra el texto procesado, y un diccionario que contiene la predicción del modelo y la probabilidad de que el texto corresponda a la clase predicha. Pueden obtenerse las tres probabiladades correspondientes a cada clase, esto al ajustar el parámetro `return_all_scores` de la clase `TextClassificationPipeline` al valor True.
Las probabilidades varían entre 0 y 1, y la suma de las tres probabilidades es igual a 1.

In [9]:
data.sample(5).values

array([['España abre fronteras para turistas vacunados, pero deja fuera a Brasil url',
        {'label': 'NEU', 'score': 0.851618230342865}],
       ['Yo era un inmigrante en los Estados Unidos cuando Obama era presidente. Como inmigrante legal, fue una de las mejores etapas por las que pude pasar. Apreciación personal y monetaria, ambiente tranquilo (a pesar de las guerras). Regresé a la elección de Trump, por miedo. Estados Unidos es mucho más grande que TRUMP',
        {'label': 'POS', 'score': 0.8571760058403015}],
       ['Si eres aguaidoniao.. En Venezuela hay mas de seis millones de colombianos los cuales se han reproducido, por lo que la cifra crece cada año. Reciben hasta mejor trato que los propios venez, Algunos han recibido beneficios de la GMVV',
        {'label': 'NEU', 'score': 0.9405502676963806}],
       ['@usuario ESTO ES PERU! PORQUE A UN VENEZOLANO SI TENEMOS NUESTROS PROPIOS DELINCUENTES CORRUPTOS TRATANDO DE ELEGIR A SU PRESIDENTE MÁS DELINCUENTE QUE ELLOS, FUJIMO